In [4]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [5]:
PATH = 'pedestrian'
NEIGHBORS = 8

In [6]:
f = open(f'{PATH}/temporalROI.txt','r') 
line = f.readline()
roi_start , roi_end = line.split() 
roi_start = int(roi_start) 
roi_end = int(roi_end)

I_start = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start)
HEIGHT, WIDTH, _ = I_start.shape

In [7]:
def gray_and_type(I):  # noqa: E741
    I = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)  # noqa: E741
    I = I.astype('uint8')  # noqa: E741
    return I

def calculate_metrics(M, ref_mask, TP, TN, FP, FN):
    TP_M = np.logical_and((M==255), (ref_mask==255))
    TP = TP + np.sum(TP_M)

    TN_M = np.logical_and((M==0), (ref_mask==255))
    TN = TN + np.sum(TN_M)

    FP_M = np.logical_and((M==255), (ref_mask==0))
    FP = FP + np.sum(FP_M)

    FN_M = np.logical_and((M==0), (ref_mask==255))
    FN = FN + np.sum(FN_M)
    return TP, TN, FP, FN

In [8]:
# Parameters
R = 20  # initial radius tolerance
N = 60  # number of background samples
T_decay = 0.5  # threshold for learning rate update
T_incr = 1  # value to increase the detection threshold
T_decr = 0.1  # value to decrease the detection threshold
T_lower = 2  # minimum detection threshold
T_upper = 200  # maximum detection threshold
bg_min = 2  # minimum number of matches to consider background
R_change = 0.05  # rate of change for R
R_lower = 18  # minimum radius tolerance
R_scale = 5  # scaling factor for radius
NEIGHBORS = 1  # number of neighbors to update

def faster_pbas(img, samples=None, param=None):
    HEIGHT, WIDTH = img.shape

    if samples is None or param is None:
        samples, param = initialize_pbas_background_model_faster(img, N)

    seg_map = np.zeros((HEIGHT, WIDTH), dtype=np.uint8)
    background = 0
    foreground = 255

    dists = np.abs(samples - img[:, :, np.newaxis])
    min_dists = np.min(dists, axis=-1)
    close_samples = dists < param[:, :, 0, np.newaxis]
    counts = np.sum(close_samples, axis=-1)

    seg_map[counts < bg_min] = foreground
    seg_map[counts >= bg_min] = background

    # Update background samples
    rand = np.random.randint(0, N, size=(HEIGHT, WIDTH))
    to_update = (rand == 0)
    rand_sample = np.random.randint(0, N, size=(HEIGHT, WIDTH))
    samples[to_update, rand_sample[to_update]] = img[to_update]

    # Update samples in the neighborhood
    xNg = (np.random.randint(-NEIGHBORS, NEIGHBORS + 1, size=(HEIGHT, WIDTH)) + np.arange(WIDTH))
    xNg = np.clip(xNg, 0, WIDTH - 1)
    yNg = (np.random.randint(-NEIGHBORS, NEIGHBORS + 1, size=(HEIGHT, WIDTH)) + np.arange(HEIGHT)[:, np.newaxis])
    yNg = np.clip(yNg, 0, HEIGHT - 1)
    rand_sample = np.random.randint(0, N, size=(HEIGHT, WIDTH))
    samples[yNg, xNg, rand_sample] = img

    # Update adaptive parameter (threshold T and radius R)
    param[:, :, 1] = np.where(min_dists < param[:, :, 0],
                              np.minimum(param[:, :, 1] + T_incr, T_upper),
                              np.maximum(param[:, :, 1] - T_decr, T_lower))

    param[:, :, 0] = np.where(param[:, :, 0] > min_dists * R_scale,
                              param[:, :, 0] * (1 - R_change),
                              param[:, :, 0] * (1 + R_change))

    return seg_map, samples, param

def initialize_pbas_background_model_faster(I_gray, N=N):
    height, width = I_gray.shape
    samples = np.zeros((height, width, N), dtype=np.uint8)
    param = np.zeros((height, width, 2), dtype=np.float32)
    param[:, :, 0] = R  # Initial detection threshold
    param[:, :, 1] = 1.0 / T_decay  # Initial learning rate

    for n in range(N):
        rand_y = np.clip(np.random.randint(-NEIGHBORS, NEIGHBORS + 1, size=(height, width)) + np.arange(height)[:, np.newaxis], 0, height - 1)
        rand_x = np.clip(np.random.randint(-NEIGHBORS, NEIGHBORS + 1, size=(height, width)) + np.arange(width), 0, width - 1)
        samples[:, :, n] = I_gray[rand_y, rand_x]

    return samples, param


In [9]:
I_start = cv2.imread(f'{PATH}/input/in%06d.jpg' % roi_start, cv2.IMREAD_GRAYSCALE)
samples = None
param = None
TP, TN, FP, FN = 0, 0, 0, 0

for i in range(roi_start+1, roi_end, 1):
    img = cv2.imread(f'{PATH}/input/in%06d.jpg' % i)
    IG = gray_and_type(img)

    ref_mask = cv2.imread(f'{PATH}/groundtruth/gt%06d.png' % i, cv2.IMREAD_GRAYSCALE)

    BG, samples, param = faster_pbas(IG, samples, param)


    TP, TN, FP, FN = calculate_metrics(BG, ref_mask, TP, TN, FP, FN)

    cv2.imshow('original', img)
    cv2.imshow('operation', samples[:,:,0])
    cv2.imshow('PBAS', BG)
    cv2.waitKey(3)



P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = 2 * P * R / (P + R)
print('PBAS precision: ', P, 'Recall: ', R, 'F1: ', F1)

PBAS precision:  0.060790659185092494 Recall:  0.9527831357268708 F1:  0.11428929037437627
